# Training Hold2Vec Embeddings

In [2]:
import os, sys, time
sys.path.append('..')

from data_loading import *
from embeddings.route_embeddings import *
from embeddings import hold2vec

2023-04-25 13:27:17.414683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
window_size = 8
embedding_size = 100
objective='skip-gram'   # 'skip-gram' or 'cbow'
epochs = 20


export_path = '../data/embeddings/'
file_name = f'hold2vec_{objective}_embedding{embedding_size}_window{window_size}_epochs{epochs}.npy'

assert os.path.exists(export_path)
print(f'Output file is {export_path + file_name}')

Saving Embeddings to ../data/embeddings/hold2vec_skip-gram_embedding100_window8_epochs20.npy


In [4]:
!pwd
!ls

/Users/benjaminglaus/Github/Moonboard/notebooks
Building Embeddings.ipynb          Training Hold2Vec Embeddings.ipynb
Data Exploration.ipynb


In [5]:
df = load_dataframe('../data/2017.json')
df.head()

,Grade,UserRating,Moves
0,6A+,1,"[H5, E7, F8, D10, E13, C14, C16, B18, E18, F9,..."
1,6A+,2,"[H5, E7, D8, D10, E13, C14, C16, B18, E18]"
2,6B+,2,"[A4, B4, C7, D9, F12, D15, F18]"
3,6A+,2,"[K4, J4, I7, H9, I10, F12, F13, E6, B14, C16, ..."
4,6B+,2,"[G8, D9, F12, C13, F15, D17, F18, E6, G4, F4]"


In [6]:
bag_of_holds_embeddings_1d = bag_of_holds_1d(df.Moves)
print(bag_of_holds_embeddings_1d.shape)

(18865, 198)


In [7]:
h2v = hold2vec.hold2vec()
h2v.initialize(N=embedding_size, window_size=window_size)

2023-04-25 13:27:25.241088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
if os.path.exists(export_path + file_name):
    print(f'File {export_path + file_name} already exists, abort training.')
else:    
    # Actual Training:
    
    start_time = time.time()
    print(f'Computing embeddings of size {embedding_size} with context window size {window_size}')
    
    h2v.build_training_data(bag_of_holds_embeddings_1d, objective=objective) # build cbow or skip-gram training data set
    h2v.build_model() # creates the keras.model
    h2v.train(epochs=epochs)  
    h2v.save_hold_matrix(filename = export_path + file_name) # write embeddings to file
    hold_embeddings = h2v.get_hold_matrix().T
    
    time_delta = time.time() - start_time
    print(f'all done in {time_delta // 60:.0f} minutes {time_delta % 60:.2f} seconds')
    print(hold_embeddings.shape)

Computing embeddings of size 100 with context window size 8
Epoch 1/20
4318/4318 [==============================] - 11s 2ms/step - loss: 3.6959 - accuracy: 0.1553
Epoch 2/20
4318/4318 [==============================] - 11s 3ms/step - loss: 3.3917 - accuracy: 0.1858
Epoch 3/20
4318/4318 [==============================] - 10s 2ms/step - loss: 3.3365 - accuracy: 0.1900
Epoch 4/20
4318/4318 [==============================] - 10s 2ms/step - loss: 3.3111 - accuracy: 0.1916
Epoch 5/20
4318/4318 [==============================] - 11s 3ms/step - loss: 3.2938 - accuracy: 0.1919
Epoch 6/20
4318/4318 [==============================] - 9s 2ms/step - loss: 3.2833 - accuracy: 0.1918
Epoch 7/20
4318/4318 [==============================] - 10s 2ms/step - loss: 3.2739 - accuracy: 0.1940
Epoch 8/20
4318/4318 [==============================] - 11s 2ms/step - loss: 3.2673 - accuracy: 0.1935
Epoch 9/20
4318/4318 [==============================] - 13s 3ms/step - loss: 3.2618 - accuracy: 0.1940
Epoch 10/20
43